In [ ]:
!wget https://fever.ai/download/fever/train.jsonl
!wget https://fever.ai/download/fever/shared_task_dev.jsonl
!wget https://fever.ai/download/fever/wiki-pages.zip

In [ ]:
!wget https://fever.ai/download/fever/shared_task_test.jsonl

In [ ]:
!wget https://fever.ai/download/fever/shared_task_test.jsonl

In [ ]:
!unzip -o wiki-pages.zip
!rm wiki-pages.zip

In [ ]:
!git clone https://github.com/Prayas-Agrawal/728-2

In [ ]:
!unzip ./728-2/finetune_data.zip

In [ ]:
!pip install faiss-cpu
!pip install sentence_transformers
!pip install evaluate

In [ ]:
import pandas as pd
import numpy as np
import os
from transformers import AutoModel, AutoTokenizer, TrainingArguments, Trainer, AutoModelForSequenceClassification
from sentence_transformers import SentenceTransformer, models, InputExample, datasets, losses
import faiss
import itertools
import torch
from tqdm.auto import tqdm
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, recall_score, precision_score, f1_score



In [ ]:
# !pip install langchain
# !pip install --upgrade --quiet langchain-text-splitters tiktoken

# !pip install huggingface_hub

In [3]:
train_df = pd.read_json("train.jsonl", lines=True)
# wiki_df = pd.read_json("train.jsonl", lines=True)
# dev_df = pd.read_json("shared_task_dev.jsonl", lines=True)

In [6]:
# test_df = pd.read_json("shared_task_test.jsonl", lines=True)
dev_df = pd.read_json("shared_task_dev.jsonl", lines=True)

In [9]:
dev_df.head()


,id,verifiable,label,claim,evidence
0,91198,NOT VERIFIABLE,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...,"[[[108548, None, None, None]]]"
1,194462,NOT VERIFIABLE,NOT ENOUGH INFO,Tilda Swinton is a vegan.,"[[[227768, None, None, None]]]"
2,137334,VERIFIABLE,SUPPORTS,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0..."
3,166626,NOT VERIFIABLE,NOT ENOUGH INFO,Anne Rice was born in New Jersey.,"[[[191656, None, None, None], [191657, None, N..."
4,111897,VERIFIABLE,REFUTES,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1..."


In [ ]:
# def make_train_df(train_df, wiki_df):
#   samples = {"claim": [], "text": []}
#   for i,r in tqdm( train_df.iterrows(), total=train_df.shape[0]):
#     ev = r["evidence"]
#     claim = r["claim"]
#     ids = []
#     for ann_data in ev:
#       for data in ann_data:
#         id = data[2]
#         if id != None and id not in ids:
#           if(id not in wiki_df.index):
#             continue
#           text = get_text_by_id(wiki_df, id)
#           ids.append(id)
#           samples["claim"].append(claim)
#           samples["text"].append(text)
#           # samples.append(InputExample(texts=[claim, text]))
#   samples = pd.DataFrame(samples)
#   return samples

In [ ]:
train_df.iloc[0]["evidence"]

In [ ]:
wiki_df.head()

In [10]:
# model_checkpoint = "distilbert-base-uncased"
# model = AutoModel.from_pretrained(model_checkpoint)
# tokenizer = AutoTokenizer.from_pretrained(model_checkpoint)
# word_embedding_model = models.Transformer('distilroberta-base')

# word_embedding_model = models.Transformer('distilbert-base-uncased')
# pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
# model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
emb_dim = 384
ret_model = SentenceTransformer("multi-qa-MiniLM-L6-cos-v1")

# test = tokenizer.encode("some sentence", truncation=True, max_length=512)
# test2 = tokenizer.batch_encode_plus(["another sentence", "another sentence"], return_tensors="pt", truncation=True, max_length=512)
# print(test2)
# ret = model(test2.input_ids).last_hidden_state

# print(ret[:, 0, :].shape)

modules.json:   0%|          | 0.00/349 [00:00<?, ?B/s]

config_sentence_transformers.json:   0%|          | 0.00/116 [00:00<?, ?B/s]

README.md:   0%|          | 0.00/11.6k [00:00<?, ?B/s]

sentence_bert_config.json:   0%|          | 0.00/53.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/612 [00:00<?, ?B/s]

...

tokenizer_config.json:   0%|          | 0.00/383 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/232k [00:00<?, ?B/s]

tokenizer.json:   0%|          | 0.00/466k [00:00<?, ?B/s]

special_tokens_map.json:   0%|          | 0.00/112 [00:00<?, ?B/s]

1_Pooling/config.json:   0%|          | 0.00/190 [00:00<?, ?B/s]

In [9]:
def mySplit(iterable, n=1):
    l = len(iterable)
    ret = []
    for ndx in range(0, l, n):
        ret.append(iterable[ndx:min(ndx + n, l)])
    return ret

def get_sentence_embed(model, text):
  # text = mySplit(text, 16)
  print("encoding num sent: ", len(text))
  emb = model.encode(text,show_progress_bar=True)
  print("embed sent shape: ", emb.shape)
  return emb

def get_sentence_embed__OLD__(text):
  text = mySplit(text, 16)
  print("text", len(text))
  emb = []
  for i, batch in enumerate(text):
    print("doing batch", i, len(batch))
    tokens = tokenizer.batch_encode_plus(batch, return_tensors="pt", truncation=True, max_length=512, padding=True)
    emb_batch = model(tokens.input_ids).last_hidden_state[:, 0, :]
    # emb.append(emb_batch)
    print("emb shape", emb_batch.shape)
    emb = emb + emb_batch.tolist()
  emb = np.asarray(emb, dtype=np.int32)
  print("embed", emb.shape)
  return emb

In [11]:
from pathlib import Path
import os.path

def load_wiki_df():
    dir = "wiki-pages"
    wiki_df = []
    for f in os.listdir(dir):
        if(f.endswith("jsonl")):
            print("loading file", f)
            wiki_df.append(pd.read_json(os.path.join(dir, f), lines=True))
    wiki_df = pd.concat(wiki_df, ignore_index=True)
    wiki_df.set_index('id', inplace=True)
    return wiki_df

def save_ids():
  dir = "wiki-pages"
  wiki_df = []
  saveDir = "./ids/"
  Path(saveDir).mkdir(parents=True, exist_ok=True)
  for f in os.listdir(dir):
    if(f.endswith("jsonl")):
      print("saving ids of file", f)
      df = pd.read_json(os.path.join(dir, f), lines=True)
      ids = df["id"]
      fPath = saveDir + f + ".csv"
      ids.to_csv(fPath, index=False)

def encode_wiki(model, prefix):
    dir = "wiki-pages"
    wiki_df = []
    emb = []
    saveDir = "./encoded_corpus/" + prefix + '/'
    Path(saveDir).mkdir(parents=True, exist_ok=True)
    for f in os.listdir(dir):
        if(f.endswith("jsonl")):
            fname = f + '.npy'
            fPath = saveDir + fname
            if os.path.isfile(fPath):
                print("already encoded", f)
                continue
            print("encoding file", f)
            df = pd.read_json(os.path.join(dir, f), lines=True)
            text = df["text"].tolist()
            emb = get_sentence_embed(model, text)
            with open(fPath, 'wb') as f:
                np.save(fPath, emb)
    print("=========DONE ENCODING=====")

      # index.add_with_ids(encoded_data, np.arange(len(data)))

wiki_df = load_wiki_df()
# save_ids()
# encode_wiki(model, "multi-qa-MiniLM-L6-cos-v1")

loading file wiki-058.jsonl
loading file wiki-092.jsonl
loading file wiki-006.jsonl
loading file wiki-069.jsonl
loading file wiki-084.jsonl
loading file wiki-035.jsonl
loading file wiki-046.jsonl
loading file wiki-102.jsonl
loading file wiki-083.jsonl
loading file wiki-049.jsonl
loading file wiki-081.jsonl
loading file wiki-043.jsonl
loading file wiki-023.jsonl
loading file wiki-091.jsonl
loading file wiki-060.jsonl
loading file wiki-022.jsonl
loading file wiki-072.jsonl
loading file wiki-039.jsonl
loading file wiki-007.jsonl
loading file wiki-066.jsonl
loading file wiki-034.jsonl
loading file wiki-109.jsonl
loading file wiki-101.jsonl
loading file wiki-026.jsonl
loading file wiki-074.jsonl
loading file wiki-042.jsonl
loading file wiki-063.jsonl
loading file wiki-047.jsonl
loading file wiki-045.jsonl
loading file wiki-021.jsonl
loading file wiki-015.jsonl
loading file wiki-050.jsonl
loading file wiki-030.jsonl
loading file wiki-048.jsonl
loading file wiki-017.jsonl
loading file wiki-05

In [ ]:
# from IPython.display import FileLink 
# FileLink('ids.zip')


In [ ]:
def get_text_by_id(df, id):
  return df.loc[id]["text"]

def get_text_by_id__LAZY__(id):
  dir = "wiki-pages"
  for f in os.listdir(dir):
    if(f.endswith("jsonl")):
      df = pd.read_json(os.path.join(dir, f), lines=True)
      _text = df.loc[df["id"] == id]["text"].values
      if _text.__len__() == 0:
        continue
      _text = _text[0]
      return _text
  print("failed to find", id)
  return ""

def make_ft_df__NEGATIVE_SAMPLED__(train_df, wiki_df, neg_sample = 10):
  ft_df = train_df.copy()
  ft_df["doc_ids"] = [[] for i in range(ft_df.shape[0])]
  ft_df["doc_text"] = [[] for i in range(ft_df.shape[0])]
  ft_df["neg_ids"] = [[] for i in range(ft_df.shape[0])]
  ft_df["neg_text"] = [[] for i in range(ft_df.shape[0])]

  for i,r in ft_df.head().iterrows():
    ev = r["evidence"]
    ids = []
    texts = []
    for ann_data in ev:
      for data in ann_data:
        id = data[2]
        if id != None:
          ids.append(id)
          texts.append(get_text_by_id(wiki_df, id))
    neg = wiki_df[~wiki_df["id"].isin(ids)].loc[:, "id"].sample(n=neg_sample).tolist()
    neg_texts = [get_text_by_id(wiki_df, neg_id) for neg_id in neg]
    ft_df.at[i,"doc_ids"] = ids
    ft_df.at[i,"doc_text"] = texts
    ft_df.at[i,"neg_ids"] = neg
    ft_df.at[i,"neg_text"] = neg_texts
  return ft_df

def make_ft_df(train_df, wiki_df):
  samples = {"claim": [], "text": []}
  for i,r in tqdm( train_df.iterrows(), total=train_df.shape[0]):
    ev = r["evidence"]
    claim = r["claim"]
    ids = []
    for ann_data in ev:
      for data in ann_data:
        id = data[2]
        if id != None and id not in ids:
          if(id not in wiki_df.index):
            continue
          text = get_text_by_id(wiki_df, id)
          ids.append(id)
          samples["claim"].append(claim)
          samples["text"].append(text)
          # samples.append(InputExample(texts=[claim, text]))
  samples = pd.DataFrame(samples)
  return samples


ft_df = make_ft_df(train_df, wiki_df)


In [ ]:
samples = []
for i, row in ft_df.iterrows():
  samples.append(InputExample(texts=[row["claim"], row["text"]]))

In [ ]:
loader = datasets.NoDuplicatesDataLoader(samples, batch_size = 32 )

In [ ]:
word_embedding_model = models.Transformer('distilbert-base-uncased')
pooling_model = models.Pooling(word_embedding_model.get_word_embedding_dimension())
model = SentenceTransformer(modules=[word_embedding_model, pooling_model])
loss = losses.MultipleNegativesRankingLoss(model)
epochs = 2
warmup = int(0.1*len(loader))
model.fit(train_objectives=[(loader, loss)], epochs = epochs, warmup_steps = warmup, output_path = "pa2_model", show_progress_bar = True, )

# FAISS

In [12]:
def index_corpus(index, emb_path):
    emb = []
    ids = []
    num2Id = {}
    currHead = 0
    for f in os.listdir(emb_path):
        if(f.endswith(".npy")):
            idFile = "/kaggle/input/wiki-ids/ids/" + f[0:-4] + ".csv"
            print("indexing file", f, idFile )
            encoded_data = np.load(os.path.join(emb_path, f))
            _ids = pd.read_csv(idFile)
            length = _ids.shape[0]
            d = {i: _ids.iloc[i-currHead].values[0]  for i in range(currHead, currHead + length)}
            num2Id.update(d)
            index.add_with_ids(encoded_data, np.arange(currHead, currHead + length))
            currHead += length
    print("=========DONE INDEXING=====")
    return num2Id
    
index = faiss.IndexIDMap(faiss.IndexFlatIP(emb_dim))
num2Id = index_corpus(index, "/kaggle/input/wiki-encoded-multi-qa-minilm-l6-cos-v1/encoded_corpus/multi-qa-MiniLM-L6-cos-v1" )



indexing file wiki-007.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-007.jsonl.csv
indexing file wiki-072.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-072.jsonl.csv
indexing file wiki-095.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-095.jsonl.csv
indexing file wiki-032.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-032.jsonl.csv
indexing file wiki-051.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-051.jsonl.csv
indexing file wiki-059.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-059.jsonl.csv
indexing file wiki-102.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-102.jsonl.csv
indexing file wiki-092.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-092.jsonl.csv
indexing file wiki-052.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-052.jsonl.csv
indexing file wiki-003.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-003.jsonl.csv
indexing file wiki-067.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-067.jsonl.csv
indexing file wiki-016.jsonl.npy /kaggle/input/wiki-ids/ids/wiki-016.jsonl.csv
indexing file wiki-024.jsonl.npy /kaggle/input/wiki-

In [14]:
def search(encoder, index, query, num2Id, wiki_df, k=1):
    query_vector = encoder.encode([query], show_progress_bar=False)
    top_k = index.search(query_vector, k)
    docs = [wiki_df.loc[num2Id[_id]].values[0] for _id in top_k[1][0]]
    lines = [wiki_df.loc[num2Id[_id]].values[1] for _id in top_k[1][0]]
    ids = [num2Id[_id] for _id in top_k[1][0]]
    return ids, docs,lines

search(ret_model, index, "How old are you?", num2Id, wiki_df, k=5)

(['How_Old_Are_You?_-LRB-song-RRB-',
  'Young_adult_-LRB-psychology-RRB-',
  'How_Old_Are_You?_-LRB-album-RRB-',
  'How_Old_Are_You?_-LRB-film-RRB-',
  'At_My_Age'],
 ["`` How Old Are You '' is a single released by Robin Gibb , the third and final single released from the album of the same name in October 1983 . It reached # 93 in the UK and # 37 in Germany . `` How Old Are You '' likewise has the rhythm guitar beat , bang-away drumming , a powerful horn and synthesizer blend . ",
  "A young adult is generally a person in the age range of 20 to 39 -LRB- or 40 -RRB- , although definitions and opinions , such as Erik Erikson 's stages of human development , vary . The young adult stage in human development precedes middle adulthood . A person in the middle adulthood stage ages from 40 -LRB- or 41 -RRB- to 64 . In old age , a person is 65 years old or older . ",
  "How Old Are You ? is the second solo album released by British rock musician Robin Gibb it released in 1983 , thirteen years 

In [41]:
dev_df.head()

,id,verifiable,label,claim,evidence
0,91198,NOT VERIFIABLE,NOT ENOUGH INFO,Colin Kaepernick became a starting quarterback...,"[[[108548, None, None, None]]]"
1,194462,NOT VERIFIABLE,NOT ENOUGH INFO,Tilda Swinton is a vegan.,"[[[227768, None, None, None]]]"
2,137334,VERIFIABLE,SUPPORTS,Fox 2000 Pictures released the film Soul Food.,"[[[289914, 283015, Soul_Food_-LRB-film-RRB-, 0..."
3,166626,NOT VERIFIABLE,NOT ENOUGH INFO,Anne Rice was born in New Jersey.,"[[[191656, None, None, None], [191657, None, N..."
4,111897,VERIFIABLE,REFUTES,Telemundo is a English-language television net...,"[[[131371, 146144, Telemundo, 0]], [[131371, 1..."


In [35]:
def calc_recall(preds, ground, k=2):
    return len(set.intersection(set(preds[:k]), set(ground)))/len(ground)

def calc_micro_ecall(preds, ground, k=2):
    return len(set.intersection(set(preds[:k]), set(ground)))/len(ground)

def calc_micro_ecall(preds, ground, k=2):
    return len(set.intersection(set(preds[:k]), set(ground)))/len(ground)


def calc_precision(preds, ground, k=2):
    return len(set.intersection(set(preds[:k]), set(ground)))/len(preds[:k])


def eval_retriever_recall(ret_model, index, num2Id, wiki_df, dataset, k=2):
    df = dataset
    recall = 0
    tot = 0
    for i, r in tqdm(df.iterrows(), total = df.shape[0]):
        preds, docs, lines = search(ret_model, index, r["claim"], num2Id, wiki_df, 5)
        ground = []
        for ann in r["evidence"]:
            for ann_data in ann:
                id = ann_data[2]
                if id != None and id not in ground:
                    if(id not in wiki_df.index):
                        continue
                    ground.append(id)
        if(len(ground) > 0):
            tot += 1
            recall += calc_recall(preds, ground)
    return recall/tot

def evaluate_nli_dev(nli_model, wiki_df, data):
    confusion_matrix = [[0,0,0] for i in range(3)]
    device = "cuda:0" if torch.cuda.is_available() else "cpu"
    nli_model = nli_model.to(device)
    with torch.no_grad():
        for i, r in tqdm(data.iterrows(), total=data.shape[0]):
            query = r["claim"]
            id = r["id"]
            evidence = r["evidence"]
            ids, docs = getWikiIdText(evidence, wiki_df)
            fin = query + " [SEP] " + " [SEP] ".join(docs)
            tok = tokenizer(fin, padding="max_length", truncation=True, max_length=512, return_tensors="pt").to(device)
            output = nli_model(**tok)
            preds = torch.nn.functional.softmax(output.logits, dim=1)
            labels = torch.argmax(preds, dim=1)[0].item()
            ground_label = label2Num[r["label"]]
            confusion_matrix[ground_label][labels] += 1
    fp_tp_sum = np.sum(confusion_matrix, axis=1)
    fn_tp_sum = np.sum(confusion_matrix, axis=0)
    tp_sum = confusion_matrix[0][0] + confusion_matrix[1][1] + confusion_matrix[2][2]
    micro_p = tp_sum/fp_tp_sum
    micro_r = tp_sum/fn_tp_sum
    micro_f1 =( 2*micro*p*micro_r)/(micro_p + micro_r)
    return micro_f1
        

In [39]:

r1 = eval_retriever(ret_model, index, num2Id, wiki_df, dev_df, k=1)
r10 = eval_retriever(ret_model, index, num2Id, wiki_df, dev_df, k=10)
r100 = eval_retriever(ret_model, index, num2Id, wiki_df, dev_df, k=100)
r100 = eval_retriever(ret_model, index, num2Id, wiki_df, dev_df, k=100)
microf1 = evaluate_nli_dev(nli_model, wiki_df, data)
print("recall@1 %.2f" % r1)
print("recall@10 %.2f" % r10)
print("recall@100 %.2f" % r100)
print("microf1 %.2f" % microf1)

recall@1 0.86
recall@10 0.57
recall@100 0.58
microf1 0.47


# NLI Model

In [18]:
label2Num = {
    "SUPPORTS": 0,
    "REFUTES": 1,
    "NOT ENOUGH INFO": 2
}

num2Label = {
    0: "SUPPORTS",
    1: "REFUTES",
    2: "NOT ENOUGH INFO"
}

nli_model = AutoModelForSequenceClassification.from_pretrained("/kaggle/input/nli-final/savedNLI/deberta-v3-base", num_labels=3,id2label=num2Label, label2id=label2Num)
tokenizer = AutoTokenizer.from_pretrained("microsoft/deberta-v3-base")


In [9]:
def getWikiIdText(evidence, wiki_df):
    ids = []
    text = []
    for ann in evidence:
        for ann_data in ann:
            id = ann_data[2]
            if id != None and id not in ids:
                ids.append(id)
                _text = wiki_df.loc[id].values[0] if id in wiki_df.index else ""
                text.append(_text)
    return ids, text

class Dataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, idx):
        claim = self.data.iloc[idx]["claim"]
        evidence = self.data.iloc[idx]["evidence"]
        ids, text = getWikiIdText(evidence, wiki_df)
        hyp = " [SEP] ".join(text)
        fin  = claim + " [SEP] " + hyp
        tokens = tokenizer(fin, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
#         item = {key: torch.tensor(val[0]) for key, val in item.items()}
        correct = label2Num[self.data.iloc[idx]["label"]]
#         labels = [0,0,0]
#         labels[correct] = 1
#         labels = torch.FloatTensor(labels)
        labels = torch.tensor(label2Num[self.data.iloc[idx]["label"]])
#         item["labels"] = labels
#         print("checking ", idx, claim, ids)
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten(), 'labels': labels}
    
    def __len__(self):
        return len(self.data)
        

In [10]:
from transformers import TrainingArguments, Trainer
import numpy as np
import evaluate

metric = evaluate.load("accuracy")
def compute_metrics(eval_pred):
    logits, labels = eval_pred
    predictions = np.argmax(logits, axis=-1)
    return metric.compute(predictions=predictions, references=labels)

# def compute_metrics(p):
#     print("computing")
#     pred, labels = p
#     pred = np.argmax(pred, axis=1)

#     accuracy = accuracy_score(y_true=labels, y_pred=pred)
#     recall = recall_score(y_true=labels, y_pred=pred)
#     precision = precision_score(y_true=labels, y_pred=pred)
#     f1 = f1_score(y_true=labels, y_pred=pred)

#     return {"accuracy": accuracy, "precision": precision, "recall": recall, "f1": f1}

training_args = TrainingArguments(output_dir="test_trainer", num_train_epochs=1, report_to="none", save_total_limit = 2, evaluation_strategy ="steps", load_best_model_at_end=True, eval_steps = 1500, save_steps=1500 )
train, test = train_test_split(train_df, test_size=0.2)
train_dataset = Dataset(train)
eval_dataset = Dataset(test)

trainer = Trainer(
    model=nli_model,
    args=training_args,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    compute_metrics=compute_metrics,
)

/opt/conda/lib/python3.10/site-packages/accelerate/accelerator.py:432: FutureWarning: Passing the following arguments to `Accelerator` is deprecated and will be removed in version 1.0 of Accelerate: dict_keys(['dispatch_batches', 'split_batches', 'even_batches', 'use_seedable_sampler']). Please pass an `accelerate.DataLoaderConfiguration` instead: 
dataloader_config = DataLoaderConfiguration(dispatch_batches=None, split_batches=False, even_batches=True, use_seedable_sampler=True)
  warnings.warn(


In [12]:
trainer.train()

Step,Training Loss,Validation Loss,Accuracy
1500,0.395900,0.329848,0.873943
3000,0.297400,0.322098,0.920488
4500,0.270700,0.279458,0.924785
6000,0.257000,0.227672,0.932245
7500,0.224100,0.233629,0.933654
9000,0.225300,0.254617,0.934548
10500,0.221500,0.214155,0.938570
12000,0.209000,0.217351,0.941251
13500,0.212700,0.211972,0.942454


TrainOutput(global_step=14545, training_loss=0.26271661533527496, metrics={'train_runtime': 17091.354, 'train_samples_per_second': 6.808, 'train_steps_per_second': 0.851, 'total_flos': 3.061616322405888e+16, 'train_loss': 0.26271661533527496, 'epoch': 1.0})

In [13]:
trainer.save_model("savedNLI/deberta-v3-base")

In [19]:
import re
from numpy import dot
from numpy.linalg import norm

def cos(a,b):
    return dot(a, b)/(norm(a)*norm(b))


def get_sentences(text):
    s = re.split(r"\d+\t", text)
    s = [s[i] for i in range(1, len(s))]
    return s

def get_best_sentences(model, query, ids, lines, k=5):
    embs = []
    idxs = []
    sents = []
    nums = []
    emb_query = model.encode([query], show_progress_bar=False)[0]
    for i,text in enumerate(lines):
        s = get_sentences(text)
        embs.extend([cos(model.encode([_s], show_progress_bar=False)[0], emb_query) for _s in s])
        sents.extend(s)
        idxs.extend([ids[i] for j in s])
        nums.extend([i for i in range(len(s))])
    amax = np.array(embs).argsort()[-k:][::-1]
    retSents = [sents[i] for i in amax]
    retIds = [[idxs[a], nums[a]] for a in amax]
    return retSents, retIds
        

In [12]:
fintest_df = test_df.copy()
fintest_df["docs"] = [[] for i in range(test_df.shape[0])]
fintest_df["predicted_evidence"] = [[] for i in range(test_df.shape[0])]


In [ ]:
for i, r in tqdm(test_df.iterrows(), total = test_df.shape[0]):
    query = r["claim"]
    ids, docs, lines = search(ret_model, index, query, num2Id, wiki_df, k=5)
#     bestSents, bestSentIds = get_best_sentences(ret_model, query, ids, lines)
    fintest_df.at[i, "docs"] = docs
#     fintest_df.at[i, "predicted_evidence"] = bestSentIds
    

In [ ]:
fintest_df.to_json(orient='records', lines=True)

In [15]:
test_df.head()

,id,claim
0,89296,Henry Spencer is played by a Greek actor.
1,78554,John Ritter died in October.
2,83809,13 Reasons Why is the only television series o...
3,49758,Playboy is a magazine.
4,22973,Alternative metal is the genre in which Alice ...


In [20]:
def tokeniseQueryWithDocs(query):
    ids, docs, lines = search(ret_model, index, query, num2Id, wiki_df, k=5)
    fin = query + " [SEP] " + " [SEP] ".join(docs)
    tok = tokenizer(fin, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
#     bestLines, bestLinesIds = get_best_sentences(ret_model, query, ids, lines)
    return tok, ids, docs, lines

def tokeniseMultipleQueryWithDocs(queries):
    fins = []
    for q in queries:
        ids, docs, lines = search(ret_model, index, q, num2Id, wiki_df, k=5)
        fin = q + " [SEP] " + " [SEP] ".join(docs)
        fins.append(fin)
    tok = tokenizer(fins, padding="max_length", truncation=True, max_length=512, return_tensors="pt")
#     bestLines, bestLinesIds = get_best_sentences(ret_model, query, ids, lines)
    return tok

class TestDataset(torch.utils.data.Dataset):
    def __init__(self, data):
        self.data = data
        
    def __getitem__(self, idx):
        claim = self.data.iloc[idx]["claim"]
        tokens = tokeniseQueryWithDocs(claim)
        return {'input_ids': tokens.input_ids.flatten(), 'attention_mask': tokens.attention_mask.flatten()}
    
    def __len__(self):
        return len(self.data)

In [158]:
test_dataset = TestDataset(test_df)


In [123]:
final_df.head()

,id,claim,predicted_label,predicted_evidence
0,89296,Henry Spencer is played by a Greek actor.,REFUTES,"[[Henry_Spencer_-LRB-disambiguation-RRB-, 3], ..."
1,78554,John Ritter died in October.,SUPPORTS,"[[James_Ritter, 7], [Charlie_Ritter, 0], [Jame..."
2,83809,13 Reasons Why is the only television series o...,REFUTES,"[[Hannah_Baker, 7], [Thirteen_Reasons_Why, 3],..."
3,49758,Playboy is a magazine.,SUPPORTS,"[[Playboy, 0], [Playboy_-LRB-lifestyle-RRB-, 1..."
4,22973,Alternative metal is the genre in which Alice ...,SUPPORTS,"[[The_Essential_Alice_in_Chains, 0], [List_of_..."


In [17]:
test_df.iloc[0:3]["claim"].values

array(['Henry Spencer is played by a Greek actor.',
       'John Ritter died in October.',
       '13 Reasons Why is the only television series of 2012 in the drama-mystery genre.'],
      dtype=object)

In [16]:
Path("./output").mkdir(parents=True, exist_ok=True)

In [ ]:
all_labels = {"index": [], "df_id":[], "claim":[], "val": [], "ids":[] ,"docs": [], "lines": []}
device = "cuda:0" if torch.cuda.is_available() else "cpu"
nli_model = nli_model.to(device)
with torch.no_grad():
    for i, r in tqdm(test_df.iterrows(), total=test_df.shape[0]):
        if(i<=13000):
            continue
        query = r["claim"]
        tok, ids, docs, lines = tokeniseQueryWithDocs(query)
        tok = tok.to(device)
        output = nli_model(**tok)
        preds = torch.nn.functional.softmax(output.logits, dim=1)
        labels = torch.argmax(preds, dim=1)[0].item()
        all_labels["index"].append(i)
        all_labels["df_id"].append(r["id"])
        all_labels["claim"].append(query)
        all_labels["ids"].append(ids)
        all_labels["docs"].append(docs)
        all_labels["lines"].append(lines)
        all_labels["val"].append(labels)
        if(i%500 == 0):
            path = "./test/all_labels_" + str(i)
            df = pd.DataFrame.from_dict(all_labels)
            df.to_csv(path, index=False)
            all_labels = {"index": [], "df_id":[], "claim":[], "val": [], "ids":[] ,"docs": [], "lines": []}
            
#         final_df.at[i, "predicted_label"] = num2Label[labels[0].item()]
#         final_df.at[i, "predicted_evidence"] = bestLinesIds
#         print(preds)
#         print(labels)
#         print(bestLinesIds)
#         print("=====")
        
#         if(i==5):
#             break

In [24]:
path = "./test/all_labels_" + str(19998)
df = pd.DataFrame.from_dict(all_labels)
df.to_csv(path, index=False)
# all_labels = {"index": [], "df_id":[], "claim":[], "val": [], "ids":[] ,"docs": [], "lines": []}

In [54]:
test_df.head()

,id,claim
0,89296,Henry Spencer is played by a Greek actor.
1,78554,John Ritter died in October.
2,83809,13 Reasons Why is the only television series o...
3,49758,Playboy is a magazine.
4,22973,Alternative metal is the genre in which Alice ...


In [60]:
pd.read_csv("/kaggle/working/test/all_labels_16000").head()
    

,index,df_id,claim,val,ids,docs,lines
0,15501,138948,Marvin Gaye's song won him a Grammy.,0,"['31st_Annual_Grammy_Awards', '22nd_Annual_Gra...","[""The 31st Annual Grammy Awards were held on F...","[""0\tThe 31st Annual Grammy Awards were held o..."
1,15502,88545,Hypertext Transfer Protocol is a standard for ...,0,"['List_of_file_transfer_protocols', 'Text-base...",['A list of notable file transfer protocols : ...,['0\tA list of notable file transfer protocols...
2,15503,205714,Capricorn is one of the four cardinal priests.,1,"['List_of_Catholic_priests', 'Luigi_Capponi', ...",['This is an incomplete list of Catholic pries...,['0\tThis is an incomplete list of Catholic pr...
3,15504,211249,Mark Strong has only been involved in American...,1,"['Mark_Henry_Sherman', 'Mark_Macias', 'Walter_...","[""Mark Henry Sherman is the founder and CEO of...","[""0\tMark Henry Sherman is the founder and CEO..."
4,15505,64526,The Judy Garland Show had 28 episodes.,1,"['The_Judy_Garland_Show', 'List_of_Judy_Garlan...","[""The Judy Garland Show was an American musica...","[""0\tThe Judy Garland Show was an American mus..."


In [21]:
fin_df = None
dir = "/kaggle/input/test-data/test"
for f in os.listdir(dir):
    temp = pd.read_csv(os.path.join(dir, f))
    if(fin_df is not None):
        fin_df = pd.concat([fin_df, temp])
    else:
        fin_df = temp


In [22]:
fin_df.shape

(13501, 7)

In [23]:
dir = "/kaggle/input/test-data-rem/test_rem/test"
for f in os.listdir(dir):
    temp = pd.read_csv(os.path.join(dir, f))
    if(fin_df is not None):
        fin_df = pd.concat([fin_df, temp])
    else:
        fin_df = temp


In [24]:
fin_df.shape

(20498, 7)

In [25]:
fin_df = fin_df.drop_duplicates(subset='df_id', keep="last")

In [26]:
fin_df.shape

(19998, 7)

In [27]:
output_done = pd.read_json("/kaggle/input/final-data/output_2024-04-04 14_29_22.jsonl", lines=True)

In [31]:
ids_list = {id: True for id in output_done["id"].tolist()}

In [33]:
Path("./output").mkdir(parents=True, exist_ok=True)

In [46]:
pd.DataFrame(fin_data).head()

,id,predicted_evidence,predicted_label
0,152153,"[[Lord's_Prayer, 4], [Lord's_Prayer, 9], [Lord...",REFUTES
1,135371,"[[Pastoralism, 16], [Pastoralist, 0], [Pastora...",SUPPORTS
2,134729,"[[Ioan_Gruffudd, 4], [Hornblower_-LRB-surname-...",SUPPORTS
3,142906,"[[Contact_-LRB-musical-RRB-, 0], [WhatsOnStage...",SUPPORTS
4,3214,"[[Alfie_-LRB-2004_film_soundtrack-RRB-, 0], [A...",REFUTES


In [ ]:
from datetime import datetime

fin_data = []
with torch.no_grad():
    for i, r in tqdm(fin_df.iterrows(), total=fin_df.shape[0]):
        df_id = r["df_id"]
        if(df_id in ids_list):
            continue
        query = r["claim"]
        ids = eval(r["ids"])
        lines = eval(r["lines"])
        label = num2Label[r["val"]]
        bestSents, bestSentIds = get_best_sentences(ret_model, query, ids, lines)
        data = {"id": df_id,"predicted_evidence": bestSentIds,"predicted_label": label}
        fin_data.append(data)
        if(i%500 == 0):
            now = datetime.now().strftime('%Y-%m-%d %H:%M:%S')
            path = "/kaggle/working/output/" + 'output_' + now + ".jsonl"
            print("saving", path)
#             df = pd.DataFrame(fin_data)
#             df.to_csv(path, index=False)
            with open(path, 'w') as outfile:
                for entry in fin_data:
                    json.dump(entry, outfile)
                    outfile.write('\n')
    


In [37]:
rem_fin = pd.DataFrame(fin_data)

In [39]:
done_df = pd.concat([output_done, rem_fin])

In [41]:
done_df.head()

,id,predicted_evidence,predicted_label
0,152153,"[[Lord's_Prayer, 4], [Lord's_Prayer, 9], [Lord...",REFUTES
1,135371,"[[Pastoralism, 16], [Pastoralist, 0], [Pastora...",SUPPORTS
2,134729,"[[Ioan_Gruffudd, 4], [Hornblower_-LRB-surname-...",SUPPORTS
3,142906,"[[Contact_-LRB-musical-RRB-, 0], [WhatsOnStage...",SUPPORTS
4,3214,"[[Alfie_-LRB-2004_film_soundtrack-RRB-, 0], [A...",REFUTES


In [43]:
done_data = []
for i,r in done_df.iterrows():
    data = {"id": r["id"],"predicted_evidence": r["predicted_evidence"],"predicted_label": r["predicted_label"]}
    done_data.append(data)
    

In [44]:
len(done_data)

19998

In [47]:
import json
path = "/kaggle/working/" + 'output_final' + ".jsonl"
with open(path, 'w') as outfile:
    for entry in done_data:
        json.dump(entry, outfile)
        outfile.write('\n')

In [ ]:
# all_labels = {}
# nli_model = nli_model.to(device)
# device = "cuda:0" if torch.cuda.is_available() else "cpu"
# with torch.no_grad():
#     batch = 64
#     for i in tqdm(range(0,test_df.shape[0], batch)):
#         queries = test_df.iloc[i:i+batch]["claim"].values
#         tok = tokeniseQueryWithDocs(queries).to(device)
#         output = nli_model(**tok)
#         preds = torch.nn.functional.softmax(output.logits, dim=1)
#         labels = torch.argmax(preds, dim=1).tolist()
#         all_labels.extend(labels)


In [40]:
import pandas as pd
import json
output = pd.read_json("/kaggle/input/preds-final/predictions.jsonl", lines=True)
test_df = pd.read_json("/kaggle/working/shared_task_test.jsonl", lines=True)


In [41]:
output.head()

,id,predicted_evidence,predicted_label
0,152153,"[[Lord's_Prayer, 4], [Lord's_Prayer, 9], [Lord...",REFUTES
1,135371,"[[Pastoralism, 16], [Pastoralist, 0], [Pastora...",SUPPORTS
2,134729,"[[Ioan_Gruffudd, 4], [Hornblower_-LRB-surname-...",SUPPORTS
3,142906,"[[Contact_-LRB-musical-RRB-, 0], [WhatsOnStage...",SUPPORTS
4,3214,"[[Alfie_-LRB-2004_film_soundtrack-RRB-, 0], [A...",REFUTES


In [9]:
test_df.head()

,id,claim
0,89296,Henry Spencer is played by a Greek actor.
1,78554,John Ritter died in October.
2,83809,13 Reasons Why is the only television series o...
3,49758,Playboy is a magazine.
4,22973,Alternative metal is the genre in which Alice ...


In [42]:
# output.loc[output['id'] == 89296]["predicted_evidence"].values[0]
output.loc[output['id'] == 168181]



,id,predicted_evidence,predicted_label
19501,168181,"[[£$€, 2], [History_of_the_London_School_of_Ec...",SUPPORTS


In [43]:
new_final = []
for i,r in test_df.iterrows():
    df_id = r["id"]
    newRow = output.loc[output['id'] == df_id]
    ev = newRow["predicted_evidence"].values[0]
    label =  newRow["predicted_label"].values[0]
    data = {"id": df_id,"predicted_evidence": ev,"predicted_label":label}
    new_final.append(data)

In [44]:
new_final[0]

{'id': 89296,
 'predicted_evidence': [['Henry_Spencer_-LRB-disambiguation-RRB-', 3],
  ['Henry_Spencer_-LRB-disambiguation-RRB-', 0],
  ['Henry_Spencer_-LRB-Psych-RRB-', 0],
  ['William_Spencer_-LRB-silent_film_actor-RRB-', 0],
  ['James_Bloor_-LRB-actor-RRB-', 0]],
 'predicted_label': 'REFUTES'}

In [46]:
path = "/kaggle/working/" + 'output_final_ordered_new' + ".jsonl"
with open(path, 'w') as outfile:
    for entry in new_final:
        json.dump(entry, outfile)
        outfile.write('\n')

In [47]:
check = pd.read_json("/kaggle/working/output_final_ordered_new.jsonl", lines=True)
check.head()

,id,predicted_evidence,predicted_label
0,89296,"[[Henry_Spencer_-LRB-disambiguation-RRB-, 3], ...",REFUTES
1,78554,"[[James_Ritter, 7], [Charlie_Ritter, 0], [Jame...",SUPPORTS
2,83809,"[[Hannah_Baker, 7], [Thirteen_Reasons_Why, 3],...",REFUTES
3,49758,"[[Playboy, 0], [Playboy_-LRB-lifestyle-RRB-, 1...",SUPPORTS
4,22973,"[[The_Essential_Alice_in_Chains, 0], [List_of_...",SUPPORTS
